## The posterior conditional of $\eta = 1 / \lambda^2$ has the density $\propto (1 + \eta) \, exp(-b\eta)$.

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf
from random import random as rand_unif
from random import expovariate as rand_exp
from random import seed as set_seed

plt.rcParams['figure.autolayout'] = False
seed = 0

In [ ]:
def compute_target_pdf(x, b):
    pdf = np.exp(- b * x) / (1 + x)
    pdf /= np.trapz(pdf, x=x)
    return pdf

Plot utility functions.

In [ ]:
figure_folder = './'

def remove_figure_box_edges(sides=['left', 'right', 'top', 'bottom']):
    for side in sides:
        plt.gca().spines[side].set_visible(False)
        
def savefig(filename, bbox_inches='tight', pad_inches=0, **kwargs):
    plt.savefig(
        figure_folder + filename, 
        bbox_inches=bbox_inches, pad_inches=pad_inches, **kwargs
    )

## Slice sampler.

In [ ]:
def update_by_slice(x, rate):
    if len(x) != len(rate):
        raise ValueError('The arguments must be of the same length.')
    NUM_STABILITY_ADJ = 2 ** - 53
    s = np.random.rand(len(x)) / (1 + x)
    upper = (1 - s) / (s + NUM_STABILITY_ADJ)
    x = sample_truncated_exponential(rate, upper)
    return x
    
def sample_truncated_exponential(rate, upper):
    """ By the inverse CDF method. """
    cum_prob = 1 - np.exp(- rate * upper)
    q = cum_prob * np.random.rand(len(rate))
    x = - log_one_minus(q) / rate
    return x
    
def log_one_minus(q):
    """ Compute log(1 - q) for q \in [0, 1]. """
    MIN_LOG_ARG = 2. ** (- 2 ** 10)
    is_small = (q < 2. ** -53)
    result = np.log(1. - q + MIN_LOG_ARG)
    result[is_small] = - q[is_small]
    return result

def generate_samples(n_sample, n_local_param, beta_tau_ratio, seed=0):
    np.random.seed(seed)
    rate = beta_tau_ratio ** 2 / 2 * np.ones(n_local_param)
    eta0 = 10. * np.random.rand(n_local_param)
    eta_samples = np.zeros((n_local_param, n_update))
    eta = eta0
    for i in range(n_update):
        eta = update_by_slice(eta, rate)
        eta_samples[:, i] = eta
    return eta_samples

In [ ]:
n_update = 10 ** 5
n_local_param = 10 ** 2
beta_gscale_ratio = np.array([.01, .001])
    
eta_samples = [
    generate_samples(
        n_update, n_local_param, ratio
    ) for ratio in beta_gscale_ratio
]

In [ ]:
plt.rcParams['font.size'] = 22
fig = plt.figure(figsize=(13, 4))


for i in range(len(beta_gscale_ratio)):
    ax = fig.add_subplot(1, 2, i + 1)
    t_series = np.log(np.squeeze(eta_samples[i][0, :]))
    plot_acf(t_series, lags=60, ax=ax, title=None)
    plt.xlabel(r'$\beta_j \, / \tau = {:1g}$'.format(beta_gscale_ratio[i]))
    plt.xticks([0, 20, 40, 60])
    tick_loc = [0, .5, 1.]
    plt.yticks(tick_loc, ['{:.1f}'.format(val) for val in tick_loc])
    if i == 0:
        plt.ylabel('Auto-correlation')
    remove_figure_box_edges(['top', 'right'])
    
fig.text(
    .5, 0, 'Lag (slice sampler)', 
    horizontalalignment='center', 
    fontsize=1.1 * plt.rcParams['font.size']
)
plt.tight_layout()
savefig('horseshoe_lscale_slice_sampler_autocorr.pdf')

plt.show()

In [ ]:
plt.figure(figsize=(13, 4))
plt.rcParams['font.size'] = 22

n_burnin = 1000
n_iter_to_plot = 1000
n_param_to_plot = 1
plt.plot(np.log(eta_samples[-1][:n_param_to_plot, n_burnin:(n_burnin + n_iter_to_plot)].T))
# plt.rcParams['mathtext.fontset'] = 'cm'
plt.ylabel(
    r'$\log(\eta)$', 
    {'fontweight': 'bold', 'fontsize': 1.1 * plt.rcParams['font.size']}
)
plt.xlabel(r'Slice sampler iterations ($\beta_j \, / \tau = {:1g}$)'.format(beta_gscale_ratio[-1]))
plt.ylim([-5.5, 16.])

remove_figure_box_edges(['top', 'right'])
plt.tight_layout()
savefig('horseshoe_lscale_slice_sampler_traceplot.pdf')
plt.show()

plt.rcParams['mathtext.fontset'] = 'dejavusans'

## Data augmentation approach.

In [ ]:
df = 1 # Degree of freedom of t-distribution.

def gibbs_update(eta, b):
    rate = 1 + df * eta
    if df == 1:
        xi = np.random.exponential(1 / rate)
    else:
        xi = np.random.gamma(.5 * (df + 1), 1 / rate)
        
    rate = df * xi + b
    if df == 1:
        eta = np.random.exponential(1 / rate)
    else:
        eta = np.random.gamma(.5 * (df + 1), 1 / rate)
    return eta, xi

def generate_samples(n_update, n_local_param, beta_gscale_ratio, seed=0):
    
    set_seed(0)
    b = beta_gscale_ratio ** 2 / 2
    eta_samples = np.zeros((n_local_param, n_update))
    xi_samples = np.zeros((n_local_param, n_update))
    eta = 10. * np.random.rand(n_local_param)
    for i in range(n_update):
        eta, xi = gibbs_update(eta, b)
        eta_samples[:, i] = eta
        xi_samples[:, i] = xi
        
    return eta_samples

In [ ]:
n_update = 10 ** 5
n_local_param = 10 ** 2
beta_gscale_ratio = [.01, .001]
    
eta_samples = [
    generate_samples(
        n_update, n_local_param, ratio
    ) for ratio in beta_gscale_ratio
]

In [ ]:
plt.rcParams['font.size'] = 22
fig = plt.figure(figsize=(13, 4))


for i in range(len(beta_gscale_ratio)):
    ax = fig.add_subplot(1, 2, i + 1)
    t_series = np.log(np.squeeze(eta_samples[i][0, :]))
    plot_acf(t_series, lags=60, ax=ax, title=None)
    plt.xlabel(r'$\beta_j \, / \tau = {:1g}$'.format(beta_gscale_ratio[i]))
    plt.xticks([0, 20, 40, 60])
    tick_loc = [0, .5, 1.]
    plt.yticks(tick_loc, ['{:.1f}'.format(val) for val in tick_loc])
    if i == 0:
        plt.ylabel('Auto-correlation')
    remove_figure_box_edges(['top', 'right'])
    
fig.text(
    .5, 0, 'Lag (data-augemented sampler)', 
    horizontalalignment='center', 
    fontsize=1.1 * plt.rcParams['font.size']
)
plt.tight_layout()
savefig('horseshoe_lscale_DA_sampler_autocorr.pdf')

plt.show()

In [ ]:
plt.figure(figsize=(13, 4))
plt.rcParams['font.size'] = 22

n_burnin = 1000
n_iter_to_plot = 1000
n_param_to_plot = 1

plt.plot(np.log(eta_samples[-1][:n_param_to_plot, n_burnin:(n_burnin + n_iter_to_plot)].T))
# plt.rcParams['mathtext.fontset'] = 'cm'
plt.ylabel(
    r'$\log(\eta)$', 
    {'fontweight': 'bold', 'fontsize': 1.1 * plt.rcParams['font.size']}
)
# plt.rcParams['mathtext.fontset'] = 'dejavusans'
plt.xlabel(r'Data-augmented sampler iterations ($\beta_j \, / \tau = {:1g}$)'.format(beta_gscale_ratio[-1]))
plt.ylim([-5.5, 16.])

remove_figure_box_edges(['top', 'right'])
plt.tight_layout()
savefig('horseshoe_lscale_DA_sampler_traceplot.pdf')
plt.show()

plt.rcParams['mathtext.fontset'] = 'dejavusans'

#### Compute the theoretical acceptance rate.

In [ ]:
def compute_accept_rate(b):
    if b > 600:
        accept_rate = 1
    else:
        L = - 2 / (1 - b) * np.log(b)
        x = np.linspace(0, L, 50001)
        bounding_pdf = np.array([
            compute_bounding_pdf(x_i, b) for x_i in x
        ])
        target_pdf = compute_target_pdf(x, b, density=False)
        accept_rate = np.trapz(target_pdf, x) / np.trapz(bounding_pdf, x)
    return accept_rate

In [ ]:
plt.figure(figsize=(6, 4))
plt.rcParams['font.size'] = 20

log10_b = np.linspace(-4., 4., 100)
accept_rate = np.array([
    compute_accept_rate(10 ** x) 
    for x in log10_b
])

plt.plot(
    log10_b, accept_rate
)
plt.axhline(1., linestyle='--', color='dimgrey')
plt.axhline(
    np.min(accept_rate), linestyle=(0, (1, 2)), color='grey',
    label='minimum'
)
plt.ylim(bottom=0)
plt.ylabel('Acceptance prob')
plt.xlabel(r'$\log_{10}(b)$')
# plt.xticks(np.linspace(-3, 3, 7))
remove_figure_box_edges(['top', 'right'])
plt.legend(loc=(.425, .3), frameon=False)

plt.tight_layout()
savefig('rejection_sampler_accept_prob.pdf')
plt.show()